<a href="https://colab.research.google.com/github/Randhir123/iisc-cds/blob/main/Randhir_M3_NB_MiniProject_3_PartA_Medical_Q%26A_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A programme by IISc and TalentSprint
### Mini-Project: Medical Q&A using GPT2

## Learning Objectives

At the end of the experiment, you will be able to:

* perform data preprocessing, EDA and feature extraction on the Medical Q&A dataset
* load a pre-trained tokenizer
* finetune a GPT-2 language model for medical question-answering

## Dataset Description

The dataset used in this project is the *Medical Question Answering Dataset* ([MedQuAD](https://github.com/abachaa/MedQuAD/tree/master)). It includes medical question-answer pairs along with additional information, such as the question type, the question *focus*, its UMLS(Unified Medical Language System) details like - Concept Unique Identifier(*CUI*) and Semantic *Type* and *Group*.

To know more about this data's collection, and construction method, refer to this [paper](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-3119-4).

The data is extracted and is in CSV format with below features:

- **Focus**: the question focus
- **CUI**: concept unique identifier
- **SemanticType**
- **SemanticGroup**
- **Question**
- **Answer**

## Part-A: Grading = 10 Points

## Information

Healthcare professionals often have to refer to medical literature and documents while seeking answers to medical queries. Medical databases or search engines are powerful resources of upto date medical knowledge. However, the existing documentation is large and makes it difficult for professionals to retrieve answers quickly in a clinical setting. The problem with search engines and informative retrieval engines is that these systems return a list of documents rather than answers. Instead, healthcare professionals can use question answering systems to retrieve short sentences or paragraphs in response to medical queries. Such systems have the biggest advantage of generating answers and providing hints in a few seconds.

### Problem Statement

Fine-tune gpt2 model on medical-question-answering-dataset for performing response generation for medical queries.

Please refer to ***M6 Assignment-1 Fine-tune GPT2*** to get familiar with how to load pre-trained gpt2 tokenizer and model.

### Import required packages

In [1]:
!pip -q install -U accelerate
!pip -q install -U transformers
!pip -q install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.8 MB/s eta 0:00:00


In [2]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

import warnings
warnings.filterwarnings('ignore')

In [3]:
#@title Download the dataset
!wget -q https://cdn.iisc.talentsprint.com/AIandMLOps/MiniProjects/Datasets/MedQuAD.csv
!ls | grep ".csv"

MedQuAD.csv


**Exercise 1: Read the MedQuAD.csv dataset**

**Hint:** pd.read_csv()

In [4]:
df = pd.read_csv("MedQuAD.csv")
df.shape

(16412, 6)

In [5]:
df.head()

,Focus,CUI,SemanticType,SemanticGroup,Question,Answer
0,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,What is (are) Adult Acute Lymphoblastic Leukem...,Key Points - Adult acute lymphoblastic leukemi...
1,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,What are the symptoms of Adult Acute Lymphobla...,"Signs and symptoms of adult ALL include fever,..."
2,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,How to diagnose Adult Acute Lymphoblastic Leuk...,Tests that examine the blood and bone marrow a...
3,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,What is the outlook for Adult Acute Lymphoblas...,Certain factors affect prognosis (chance of re...
4,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,Who is at risk for Adult Acute Lymphoblastic L...,Previous chemotherapy and exposure to radiatio...


### Pre-processing and EDA

**Exercise 2: Perform below operations on the dataset [0.5 Mark]**

- Handle missing values
- Remove duplicates from data considering `Question` and `Answer` columns

- **Handle missing values**

In [6]:

df.dropna(inplace=True)

In [7]:
df.shape

(15810, 6)

In [8]:
# Drop missing values



- **Remove duplicates from data considering `Question` and `Answer` columns**

In [9]:
# Check duplicates

df.drop_duplicates(subset=['Question', 'Answer'], inplace=True)

In [10]:
# Drop duplicates

df.shape

(15762, 6)

In [11]:
# Check duplicates


**Exercise 3: Display the category name, and the number of records belonging to top 100 categories of `Focus` column [1 Mark]**

In [12]:

category_counts = df['Focus'].value_counts()
sorted_categories = category_counts.sort_values(ascending=False)

In [13]:
# Top 100 Focus categories names

top_100_categories = sorted_categories.head(100)
print("Top 100 Categories of Focus column:")
for category, count in top_100_categories.items():
    print(f"Category: {category}, Count: {count}")

Top 100 Categories of Focus column:
Category: Breast Cancer, Count: 53
Category: Prostate Cancer, Count: 43
Category: Stroke, Count: 35
Category: Skin Cancer, Count: 34
Category: Alzheimer's Disease, Count: 30
Category: Lung Cancer, Count: 29
Category: Colorectal Cancer, Count: 29
Category: High Blood Cholesterol, Count: 28
Category: Heart Failure, Count: 28
Category: Heart Attack, Count: 28
Category: High Blood Pressure, Count: 27
Category: Parkinson's Disease, Count: 25
Category: Leukemia, Count: 22
Category: Osteoporosis, Count: 21
Category: Shingles, Count: 21
Category: Age-related Macular Degeneration, Count: 20
Category: Diabetes, Count: 20
Category: Hemochromatosis, Count: 20
Category: Gum (Periodontal) Disease, Count: 19
Category: Psoriasis, Count: 19
Category: Diabetic Retinopathy, Count: 19
Category: Kidney Disease, Count: 17
Category: Cataract, Count: 16
Category: Dry Mouth, Count: 16
Category: COPD, Count: 16
Category: Balance Problems, Count: 16
Category: Wilson Disease, C

### Create Training and Validation set

**Exercise 4: Create training and validation set [2 Marks]**

- Consider 4 samples per `Focus` category, for each top 100 categories, from the dataset (It will give 400 samples for training)

- Consider 1 sample per `Focus` category (different from training set), for each top 100 categories, from the dataset (It will give 100 samples for validation)

In [14]:

top_100_categories = df['Focus'].value_counts().head(100).index
df_top_100 = df[df['Focus'].isin(top_100_categories)]

training_samples = df_top_100.groupby('Focus').apply(lambda x: x.sample(n=4)).reset_index(drop=True)
validation_samples = df_top_100.groupby('Focus').apply(lambda x: x.drop_duplicates(subset=['Focus']).sample(n=1)).reset_index(drop=True)

training_set = training_samples.reset_index(drop=True)
validation_set = validation_samples.reset_index(drop=True)

print("Training set shape:", training_set.shape)
print("Validation set shape:", validation_set.shape)

Training set shape: (400, 6)
Validation set shape: (100, 6)


### Pre-process `Question` and `Answer` text

**Exercise 5: Perform below tasks: [1.5 Marks]**

- Combine `Question` and `Answer` for train and validation data as shown below:
    - sequence = *'\<question\>' + question-text + '\<answer\>' + answer-text*

- Join the combined text using '\n' into a single string for training and validation separately

- Save the training and validation strings as separate text files

- **Combine Question and Answer for train and val data**

In [15]:

training_combined = training_set.apply(lambda row: f'<question>{row["Question"]}<answer>{row["Answer"]}', axis=1)
training_combined_text = '\n'.join(training_combined)

validation_combined = validation_set.apply(lambda row: f'<question>{row["Question"]}<answer>{row["Answer"]}', axis=1)
validation_combined_text = '\n'.join(validation_combined)

- **Join the combined text using '\n' into a single string for training and validation separately**

- **Save the training and validation strings as text files**

In [17]:

with open('training_data.txt', 'w') as f:
    f.write(training_combined_text)

with open('validation_data.txt', 'w') as f:
    f.write(validation_combined_text)

**Exercise 6: Load pre-trained GPT2Tokenizer [0.5 Mark]**

- Use checkpoint = "gpt2"

In [18]:

checkpoint = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

**Exercise 7: Tokenize train and validation data and form TextDataset objects [0.5 Mark]**

- Use the loaded pre-trained tokenizer
- Use training and validation data saved in text files

In [19]:

# Tokenize train text
train_dataset = TextDataset(tokenizer=tokenizer, file_path="training_data.txt", block_size=128)

# Tokenize validation text
val_dataset = TextDataset(tokenizer=tokenizer, file_path="validation_data.txt", block_size=128)

**Exercise 8: Create a DataCollator object [0.5 Mark]**

In [20]:

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="pt")

**Exercise 9: Load pre-trained GPT2LMHeadModel [0.5 Mark]**

In [21]:

model = GPT2LMHeadModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

**Exercise 10: Fine-tune GPT2 Model [1 Mark]**

- Specify training arguments and create a TrainingArguments object (Use 30 epochs)

- Train a GPT-2 model using the provided training arguments

- Save the resulting trained model and tokenizer to a specified output directory

In [22]:
# Set up the training arguments

model_output_path = "/content/gpt_model"

training_args = TrainingArguments(
    output_dir = model_output_path,
    overwrite_output_dir = True,
    per_device_train_batch_size = 4, # try with 2
    per_device_eval_batch_size = 4,  #  try with 2
    num_train_epochs = 100,
    save_steps = 1_000,
    save_total_limit = 2,
    logging_dir = './logs',
    )

In [23]:
# Train the model
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
)

trainer.train()

# Save the model
trainer.save_model(model_output_path)

# Save the tokenizer
tokenizer.save_pretrained(model_output_path)

Step,Training Loss
500,2.573100
1000,2.026500
1500,1.680300
2000,1.387800
2500,1.125300
3000,0.933600
3500,0.765100
4000,0.619500
4500,0.503400
5000,0.416400


('/content/gpt_model/tokenizer_config.json',
 '/content/gpt_model/special_tokens_map.json',
 '/content/gpt_model/vocab.json',
 '/content/gpt_model/merges.txt',
 '/content/gpt_model/added_tokens.json')

**Exercise 11: Test Model with user input prompts [1 Mark]**

- Create `generate_response()` function that takes a trained *model*, *tokenizer*, and a *prompt* string as input and generates a response using the GPT-2 model

- Test it with some user input prompts

In [38]:
# Load the fine-tuned model and tokenizer

my_model = GPT2LMHeadModel.from_pretrained(model_output_path)
my_tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

In [37]:
# Response from model

def generate_response(model, tokenizer, prompt, max_length=100):

    input_ids = tokenizer.encode(prompt, return_tensors="pt")      # 'pt' for returning pytorch tensor

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [39]:
# Testing with given prompt 1

prompt = "What to do when feeling sick?"
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: What to do when feeling sick??<answer>People who have chronic illnesses that require them to use a medicine such as antibiotics, radiation, chemotherapy, or a heart transplant may find it helpful to learn about the medicines they take. There are also some natural treatments, taken by mouth or applied to the skin, that can help relieve the symptoms of sickness. There are also some natural treatments, taken by mouth or applied to the skin, that can help relieve the symptoms of sickness.



In [40]:
# Testing with given prompt 2

prompt = "What precautions to take for a healthy life?"
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: What precautions to take for a healthy life? Read more about how people are cared for at the heart of heart disease.
<question>What is (are) Heart Failure?<answer>If you have heart failure, learn about the severity of the problem and how to care for someone who has it. Talk with people who have heart failure and ask how you can learn more about their lives. Answer honestly if a doctor or other health care professional asks you about other heart problems. Have a honest


**Exercise 12: Compare the performance of a *GPT2 model* with the *GPT2 model fine-tuned* on MedQuAD data [1 Mark]**

- Load another pre-trained GPT2LMHeadModel and do not fine-tune it

- To generate response using the untuned model, pass it as a parameter to `generate_response()` function

- Test both models (fine-tuned and untuned) with below user input prompts:

    - "What precautions to take for a healthy life?"
    - "What to do after being diagnosed with cancer?"
    - "What to do when feeling sick?"

In [41]:
# Load a pre-trained GPT2 model, do not finetune it with MedQuAD data

model_gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")

In [42]:
# Testing with finetuned model: prompt 1

prompt = "What to do when feeling sick?"
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: What to do when feeling sick??<answer>People who have chronic illnesses that require them to use a medicine such as antibiotics, radiation, chemotherapy, or a heart transplant may find it helpful to learn about the medicines they take. There are also some natural treatments, taken by mouth or applied to the skin, that can help relieve the symptoms of sickness. There are also some natural treatments, taken by mouth or applied to the skin, that can help relieve the symptoms of sickness.



In [43]:
# Testing with untuned model: prompt 1

prompt = "What to do when feeling sick?"
response = generate_response(model_gpt2, tokenizer_gpt2, prompt)
print("Generated response:", response)

Generated response: What to do when feeling sick?

The first thing you should do is to get your body to relax.

If you're feeling sick, you should take a few minutes to relax.

If you're feeling sick, you should take a few minutes to relax.

If you're feeling sick, you should take a few minutes to relax.

If you're feeling sick, you should take a few minutes to relax.

If you're feeling sick, you


In [44]:
# Testing with finetuned model: prompt 2

prompt = "What precautions to take for a healthy life?"
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: What precautions to take for a healthy life? Read more about how people are cared for at the heart of heart disease.
<question>What is (are) Heart Failure?<answer>If you have heart failure, learn about the severity of the problem and how to care for someone who has it. Talk with people who have heart failure and ask how you can learn more about their lives. Answer honestly if a doctor or other health care professional asks you about other heart problems. Have a honest


In [45]:
# Testing with untuned model: prompt 2

prompt = "What precautions to take for a healthy life?"
response = generate_response(model_gpt2, tokenizer_gpt2, prompt)
print("Generated response:", response)

Generated response: What precautions to take for a healthy life?

The following are some of the most common questions you'll hear from your doctor or nurse about your health.

What are the risks of taking a drug that can cause cancer?

The risks of taking a drug that can cause cancer are very high.

What are the risks of taking a drug that can cause cancer?

The risks of taking a drug that can cause cancer are very high.

What are the risks


In [46]:
# Testing with finetuned model: prompt 3

prompt = "What to do after being diagnosed with cancer?"
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)


Generated response: What to do after being diagnosed with cancer? - Talk with your doctor about talking with a dietitian who can help you plan a healthy diet, and about doing exercises that can help you regain muscle tone. - Have your doctor check any changes in your eating and drinking to make sure you are getting the best results from your new treatment. - Have your doctor monitor your cholesterol levels and take any medicines you take. - Answer honestly if a doctor or other health care professional asks you about other drug or


In [47]:
# Testing with untuned model: prompt 3

prompt = "What to do after being diagnosed with cancer?"
response = generate_response(model_gpt2, tokenizer_gpt2, prompt)
print("Generated response:", response)

Generated response: What to do after being diagnosed with cancer?

The first step is to get your doctor's approval for a treatment.

If you have a cancer diagnosis, you may need to get a second opinion.

If you have a cancer diagnosis, you may need to get a second opinion. If you have a cancer diagnosis, you may need to get a third opinion.

If you have a cancer diagnosis, you may need to get a third opinion. If you have a cancer
